# Restful—分页和过滤

In [ ]:
# settings.py 中
# 配置restful api 返回结果

REST_FRAMEWORK = {
    # 分页
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 8,
    # 设置搜索 想要实现这个功能需要pip install djangorestframework==3.4.6,一定要注意版本
    'DEFAULT_FILTER_BACKENDS':('rest_framework.filters.DjangoFilterBackend',
                               'rest_framework.filters.SearchFilter'),
    # 返回结构自定义
    'DEFAULT_RENDERER_CLASSES': (
        'utils.renderResponse.CustomJsonRenderer',
    )
}

In [ ]:
# models.py中
# 以下是创建的数据库表
from django.db import models


# Create your models here.


class Student(models.Model):

    s_name = models.CharField(max_length=10)
    s_tel = models.CharField(max_length=11)
    s_yuwen =models.DecimalField(max_digits=3, decimal_places=1, null=True)
    s_operate_time = models.DateTimeField(null=True)
    STATUS = [
        ('NONE','正常'),
        ('NEXT_SCH', '留级'),
        ('DROP_SCH', '退学'),
        ('LEAVE_SCH', '休学')
    ]
    # choices表示只能取STATUS中的数据，毕竟choices内部要求只能接受list和tuple类型的数据
    s_status = models.CharField(max_length=20,choices=STATUS,default='NONE')
    s_delete = models.BooleanField(default=0)

    class Meta:
        db_table = 'day5_student'


In [ ]:
# 在项目app文件夹下，新建一个filters.py文件
import django_filters
from rest_framework import filters

from stu.models import Student


class StuFilter(filters.FilterSet):
    # 如果不加lookup_expr就是精确搜索，加上之后是模糊搜索
    # icontains表示是忽略的发源地
    name = django_filters.CharFilter('s_name', lookup_expr='icontains')
    tel = django_filters.CharFilter('s_tel')
    status = django_filters.CharFilter('s_status')
    # gt 表示筛选出来的数据大于传入的参数
    operate_time_min = django_filters.DateTimeFilter('s_operate_time', lookup_expr='gt')
    operate_time_max = django_filters.DateTimeFilter('s_operate_time', lookup_expr='lt')
    yuwen_min = django_filters.NumberFilter('s_yuwen', lookup_expr='gte')
    yuwen_max = django_filters.NumberFilter('s_yuwen', lookup_expr='lte')

    class Meta:
        model=Student
        # 新版本的filter不需要再fields里添加数据
        # 以前的旧版本需要添加准备过滤的字段
        # fields = ['s_name', 's_tel', 's_status', 's_operate_time', 's_yuwen']
        fields = []

In [ ]:
# 在项目app文件夹下，新建一个serializers.py文件
from rest_framework import serializers

from stu.models import Student


class StudentSerializer(serializers.ModelSerializer):
    #  可以在restful中对传递给数据库的进行过滤
    s_name = serializers.CharField(error_messages={
        'blank': '用户名不能为空',
        'max_length': '用户名不能超过10个字符'
    }, max_length=10)
    s_tel = serializers.CharField(error_messages={
        'blank': '电话不能为空'
    })
    class Meta:
        model = Student
        #  下面是要展示的字段
        fields = ['id', 's_name', 's_tel','s_yuwen', 's_operate_time', 's_status']

    def to_representation(self, instance):

        data = super().to_representation(instance)
        try:
            data['s_addr'] = instance.studentinfo.i_addr
        except Exception as e:
            data['s_addr'] = ''
        # 将s_status中的英文转化成对应的中文
        data['s_status'] = dict(Student.STATUS)[data['s_status']]

        return data

In [ ]:
# urls.py文件中
from django.conf.urls import url
from django.contrib.auth.decorators import login_required

from rest_framework.routers import SimpleRouter

from stu import views

router = SimpleRouter()
# 实例化一个路由，将要访问的路由注册进去，这种方式调用的是一个类
router.register(r'student', views.StudentEdit)

urlpatterns = [

]
urlpatterns += router.urls

In [ ]:
# views.py中
# 继承这几个个类之后，就不需要写增删改查
class StudentEdit(mixins.ListModelMixin,  # 获取所有数据
                  mixins.RetrieveModelMixin,  # 获取某一个数据
                  mixins.UpdateModelMixin,  # 更新数据，可以使用put和patch
                  mixins.DestroyModelMixin,  # 删除数据，可以是用delete
                  mixins.CreateModelMixin,  # 创建数据，可以使用post方法
                  viewsets.GenericViewSet):

    # 查询所有信息
    queryset = Student.objects.all()
    # 序列化
    serializer_class = StudentSerializer
    # 过滤
    filter_class = StuFilter

    # 重写排序的的条件
    def get_queryset(self):

        query = self.queryset
        # 在这里只通过s=delete=0的数据
        return query.filter(s_delete=0).order_by('-id')

    # 重写了删除的方法，删除时不直接删除数据，而是将s_delete的值修改为1
    def destroy(self, request, *args, **kwargs):
        instance = self.get_object()
        instance.s_delete = 1
        instance.save()
        return Response({'msg':'删除成功', 'code': 200})